# Demo Notebook to train, save, and load a model, which will be used on the Flask Webapp

# Problem Statement
To predict the presence of perceptual brand attributes in the images that consumers post online. 

## Multiclass Problem
It is a multiclass problem with a brand can be preceived with one of the 4 attributes : Fun, Healthy, Rugged or Glamarous. 
For current problem, we are follwing transfer learning approach to use pretrained resnet50 model and retrain the final layer 
with our the training data.
For solving multiclass problems, two approaches are used in machine learning

### One Vs Rest (One Vs All)
Requires n classifiers if n number of classes exist. **Decision rule**:Predict class label with the highest probability. 


### One Vs One
In this we have to train binary classifier for each class pair. 
**Decision rule**:Score for output a data item towards one class, combines all classifier's probability involving this class in the class pairs. Requires nC2 classfiers if n number of classes exist.

For current problem, we are choosing One Vs Rest approach as we want to limit the number of classifiers to number of attributes. Each classifier requires its own dataset, and loading multiple datasets into memory with huge trainign corpus results in memory not avaliable errors. 

## Importing Libraries

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
try: 
  print('set memory growth')
  tf.config.experimental.set_memory_growth(physical_devices[0], True) 
except: 
  # Invalid device or cannot modify virtual devices once initialized. 
  pass 

In [ ]:
#from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle


import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
import keras
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers


from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import matplotlib.pyplot as plt
import cv2
import os


In [ ]:
IMG_WIDTH=300
IMG_HEIGHT=300
IMG_DIM = (IMG_WIDTH, IMG_HEIGHT)

## Data preprocessing
We are going to do following data processing steps

 - Read images in 2 arrays corresponding to positive or negative images
 - Remove corrupt images
 - label all the antonym negative on all attributes
 - Since we already have large amount of training images, we are not going to use any data augmentation technique.


In [ ]:
# Process all images in the attribute folders
# path to the downloaded flickr images
data_dir = 'C://Users//neera//Documents//Python Scripts//2nd_sem//DLS//data_code_submission//data//Flickr//images_train//'
# We are looking for 4 positive and 4 negative attiribures for training the model
all_attributes = ['glamorous', 'rugged', 'fun', 'healthy']
all_antonyms = ['drab', 'gentle', 'unhealthy', 'dull']
num_attributes = len(all_attributes)

n_images = []
history = [[] for i in range(num_attributes)]
p_images = [[] for i in range(num_attributes)]
pi_array = [[] for i in range(num_attributes)]
train_class = [[] for i in range(num_attributes)]
y_labels = [[] for i in range(num_attributes)]

label = 'negative'
for i in range(num_attributes):
    antonym_dir = data_dir + all_antonyms[i]
    r_images = [antonym_dir + '//' + f for f in os.listdir(antonym_dir)]
    j = 0
    for r in r_images:
        img = cv2.imread(r)
        if img is None: # We only keep good images
            continue
        n_images.append(r)
        #all_labels.append(label)
        j = j+1
        if j > 3:
            break
        

# For each attribute, assign a label to the image. 
for i in range(num_attributes):
    attribute_dir = data_dir + all_attributes[i]
    r_images = [attribute_dir + '//' + f for f in os.listdir(attribute_dir)]
    
    label = all_attributes[i]
    j = 0
    for r in r_images:
        img = cv2.imread(r)
        if img is None: # We only keep good images
            continue
        p_images[i].append(r)
        #all_labels.append(label)
        j = j+1
        if j > 10:
            break

## p_image
This structure consists of names of all iamges corresponding to positive attributes. We are going to work with images name for data preprocessing, instead of working with image data itself. Since image data is quite a big strucutre, working with just image name,  we can make certain preprocessing functions to execure faster

In [ ]:
for i in range(num_attributes):
    pi_array[i] = np.array(p_images[i])
    pi_array[i] =  pi_array[i].reshape(pi_array[i].shape[0],1)
 
ni_array =np.array(n_images)
ni_array = ni_array.reshape(ni_array.shape[0],1)


# Creating four copies of Training Data

Since we are using one vs rest approach of multiclass classification, we need to create 4 classifiers one for each attribute.  We need to create 4 datasets, one for each classifier. 

 - First DataSet will have samples with attribute Glamorous being labeled as 1, and all other images are labeled as negative. 

 - Second DataSet will have samples with attribute Rugged being labeled as 1, and all other images are labeled as negative. 

 - Basically each dataset will have identify one of the class as positive(labeled as 1) and all other images as negative(labeled as 0)

In [ ]:
# 4 copies of training data set each corresponding to one classfier
train_class[0] = np.vstack((pi_array[0], pi_array[1],pi_array[2], pi_array[3], ni_array))
train_class[1] = np.vstack((pi_array[1], pi_array[0],pi_array[2], pi_array[3], ni_array))
train_class[2] = np.vstack((pi_array[2], pi_array[0],pi_array[1], pi_array[3], ni_array))
train_class[3] = np.vstack((pi_array[3], pi_array[0],pi_array[1], pi_array[2], ni_array))

print(train_class[0].shape)
print(train_class[1].shape)
print(train_class[2].shape)
print(train_class[3].shape)

y_labels[0] = np.vstack((np.ones((pi_array[0].shape[0],1)) , 
                         np.zeros((pi_array[1].shape[0] +pi_array[2].shape[0] + pi_array[3].shape[0] + ni_array.shape[0] ,1))))

y_labels[1] = np.vstack((np.ones((pi_array[1].shape[0],1)) , 
                         np.zeros((pi_array[0].shape[0] +pi_array[2].shape[0] + pi_array[3].shape[0] + ni_array.shape[0] ,1))))

y_labels[2] = np.vstack((np.ones((pi_array[2].shape[0],1)) , 
                         np.zeros((pi_array[0].shape[0] +pi_array[1].shape[0] + pi_array[3].shape[0] + ni_array.shape[0] ,1))))

y_labels[3] = np.vstack((np.ones((pi_array[3].shape[0],1)) , 
                         np.zeros((pi_array[0].shape[0] +pi_array[1].shape[0] + pi_array[2].shape[0] + ni_array.shape[0] ,1))))

print(y_labels[0].shape)
print(y_labels[1].shape)
print(y_labels[2].shape)
print(y_labels[3].shape)

## Train-Test Split 
 - We split the data set into training,validation and testing data in ration of 80,10 and 10 percent
 - We are still working with images name instead of image data itself, as working with images names(which is much smaller strucutre than image data) make pre processing much more faster. 
 - Since we are not doing any data augmentation, we can load the actual image after all the data preprocessing.

In [ ]:
def train_data_split(all_images,all_labels):
    
    num_images = len(all_images)
    shuffled_all_images, shuffled_all_labels, shuffled_id = \
    shuffle(all_images, all_labels, list(range(num_images)), random_state=66)

    # As the images/labels are shuffled in the previous stage, they are 
    # randomly ordered, so we will truncated the data directly into train, val, 
    # and test

    num_images = len(shuffled_all_images)
    # number of 10% of all images
    num_10_percent = int(num_images * 0.1)
    print(num_10_percent)

    train_80_images = shuffled_all_images[0:num_10_percent * 8]
    train_labels = shuffled_all_labels[0:num_10_percent * 8]

    val_10_images = shuffled_all_images[num_10_percent*8:num_10_percent*9]
    val_labels = shuffled_all_labels[num_10_percent*8:num_10_percent*9]

    test_10_images = shuffled_all_images[num_10_percent*9:]
    test_labels = shuffled_all_labels[num_10_percent*9:]

    print(len(train_80_images))
    print(len(train_labels))
    print(len(val_10_images))
    print(len(val_labels))
    print(len(test_10_images))
    print(len(test_labels))
    return (train_80_images, train_labels,val_10_images, val_labels,test_10_images,test_labels)

X_train = [[] for i in range(num_attributes)]
X_val = [[] for i in range(num_attributes)]
X_test = [[] for i in range(num_attributes)]
y_train = [[] for i in range(num_attributes)]
y_val = [[] for i in range(num_attributes)]
y_test = [[] for i in range(num_attributes)]
for i in range(num_attributes):
    print(i)
    (X_train[i], y_train[i],X_val[i], y_val[i],X_test[i],y_test[i]) = train_data_split(train_class[i], y_labels[i])


## Merge Test data for all attributes

Test data needs to be common to all the 4 classifiers. For test data, prediction from each classifier will be merged to give one label for each test sample. Suppose for a paricular test image, classifier 1 gives probability(0.1, prediction is that image does not belong to first attribute), classifier 2 gives probability (0.8, prediction is that image does belong to 2nd attribute), and so on. All the four predictions will be merge together to give a label correponding to a classifier with highest probability. 

In [ ]:

index_0 = np.where(y_test[0] == 1)
count_0 =  (y_test[0][index_0]).shape[0]
index_1 = np.where(y_test[1] == 1)
count_1 =  (y_test[1][index_1]).shape[0]
index_2 = np.where(y_test[2] == 1)
count_2 =  (y_test[2][index_2]).shape[0]
index_3 = np.where(y_test[3] == 1)
count_3 =  (y_test[3][index_3]).shape[0]
y_test_all = np.vstack((np.zeros((count_0, 1)), np.ones((count_1,1)), 2*np.ones((count_2,1)), 3*np.ones((count_3,1)) ))
X_test_all = []
for j in index_0[0]:
    X_test_all.append(X_test[0][j])
for j in index_1[0]:
    X_test_all.append(X_test[1][j])
for j in index_2[0]:
    X_test_all.append(X_test[2][j])
for j in index_3[0]:
    X_test_all.append(X_test[3][j])
X_test_all = np.array(X_test_all)

print(y_test_all.shape)
print(X_test_all.shape)



## Save training, validation and test data
Since training data is very large, we cannot load all 4 datasets at the same time. GPU memory allows only 1 dataset to be loaded We need to train each classifier independently. So we save the training/validation/test data into numpy arrays, traing each classifier one by one by loading corresponding data set. 

In [ ]:
for i in range(num_attributes):
    train_images_file = '5000//X_train_' + str(i)
    np.save(train_images_file, X_train[i])
    train_label_file = '5000//y_train_' + str(i)
    np.save(train_label_file, y_train[i])
for i in range(num_attributes):
    val_images_file = '5000//X_val_' + str(i)
    np.save(val_images_file, X_val[i])
    val_label_file = '5000//y_val_' + str(i)
    np.save(val_label_file, y_val[i])
np.save('5000//X_test_all', X_test_all)
np.save('5000//y_test_all', y_test_all)
